In [ ]:
import http.client
import json
import os
import re
import requests
import pyperclip
import sys

from bs4 import BeautifulSoup
from getpass import getpass
from time import sleep, time as now
from urllib.parse import urljoin
from urllib.request import urlopen

In [ ]:
# Set this to 1 for debugging HTTP server communication
http.client.HTTPConnection.debuglevel = 0

In [ ]:
bamboo_base_url = "https://bamboo.ec2.local"
local_dir = os.path.expanduser('./assets/traceability') # local directory where all reports are stored
security_certificate = os.path.expanduser("~/Americas-Root.pem")
# ec2_username = input('user: ')
# ec2_password = getpass()
ec2_username = "automatedservices"
ec2_password = "EC2RO-ResMed123+"

In [ ]:
def last_successful_build():
    """Gets the build number of the last successful build"""
    response = requests.get('https://bamboo.ec2.local/browse/MON-ATEST',
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    soup = BeautifulSoup(markup=response.content.decode(), features="html")
    table = soup.find("table", { "id" : "buildResultsTable" })
    table_body = table.find('tbody')
    return next(node.find('a').text.strip()[1:] 
                for row in table_body.find_all('tr')
                for node in (row.find('td', {'class': 'Successful'}),) if node)

In [ ]:
last_successful_build()

In [ ]:
def download_all_reports(build_number=None):
    """Downloads all report.json files for a given build number."""
    # https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-2059/
    build_path = f"artifact/MON-ATEST/shared/build-{build_number or last_successful_build()}/"
    bamboo_build_url = urljoin(bamboo_base_url, build_path)
    response = requests.get(bamboo_build_url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    refs = extract_links_from_contents(response.content)
    for ref in refs:
        download_report(urljoin(bamboo_base_url, ref))

In [ ]:
def extract_links_from_contents(contents):
    """Return a list of URLs found in given contents."""
    soup = BeautifulSoup(markup=contents, features="html")
    links = soup.find_all("a")
    return [
        link['href'] for link in links
        if re.match(r'.*Acceptance-Test-Results-\d+', str(link.contents))
    ]

In [ ]:
def download_report(results_url):
    """Downloads and stores a report.json file from given URL"""
    target_dir = os.path.join(local_dir, get_local_sub_dir(results_url))
    os.makedirs(target_dir, exist_ok=True)
    url = f'{results_url}/cucumberReports/report.json'
    print(f"Downloading {url} ...", end='')

    response = requests.get(url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    with open(os.path.join(target_dir, 'report.json'), mode='wb') as f:
        f.write(response.content)
    print('done!')

In [ ]:
def get_local_sub_dir(url):
    "Returns the local sub-directory to store report downloaded from given URL."
    m = re.match(r'.*/(build-\d+/Acceptance-Test-Results-\d+)/?', url)
    return m.group(1) if m else None

In [ ]:
download_all_reports()

In [ ]:
def get_local_reports(build_number=None):
    """Returns the file path of all locally stored reports.json files 
    that belong to a build.
    
    If build_number is None, then the build number defaults to the last
    successful build"""
    root = os.path.join(local_dir, f'build-{build_number or last_successful_build()}')
    result = []
    for dir_path, dir_names, file_names in os.walk(root):
        if file_names and 'report.json' in file_names:
            result.append(os.path.join(dir_path, 'report.json'))
    result.sort()
    return result

In [ ]:
get_local_reports()

In [ ]:
def post_all_to_pipeline(build_number=None):
    build_number = build_number or last_successful_build()
    report_files = get_local_reports(build_number=build_number)
    if not report_files:
        download_all_reports(build_number)
        report_files = get_local_reports(build_number=build_number)
    for file_name in get_local_reports(build_number=build_number):
        print(f'Transforming: {file_name}...', end='')
        response = post_to_transform(file_name)
        print(response.status_code)
        if response.status_code != 200:
            break
        transformed_json = { 
            "projectKey": "MCA",
            "fixVersion": "Pacific TBD",
            "testCycle": "Pacific Test", 
            **response.json()
        }
        print('Posting to update...', end='')
        response = post_to_update(transformed_json)
        print(response.status_code)
        if response.status_code != 200:
            break
        yield response

In [ ]:
def post_to_transform(file_name):
    transform_url = 'https://taas-traceability-service.apps.us6.paas.easycareonline.com/taas/traceability/report/transform'
    headers = {'Content-Type': 'multipart/form-data', 'Accept': 'application/json'}
    multipart_form_data = {'file': ('report.json', open(file_name, 'rb'), 'application/json', headers)}
    return requests.post(transform_url, files=multipart_form_data, verify=security_certificate)

In [ ]:
def post_to_update(transformed_json):
    update_url = 'https://taas-traceability-service.apps.us6.paas.easycareonline.com/taas/traceability/report/update'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    return requests.post(update_url, headers=headers, data=json.dumps(transformed_json) , verify=security_certificate)

Generate all the cards for a build:

In [ ]:
results = list(post_all_to_pipeline())

Let's save the results:

In [ ]:
with open(f'assets/traceability/build-2081/results.json', mode='w') as f:
    json.dump([result.json() for result in results], f, separators=[',', ':'])